In [1]:
import os
import csv
from lib.crawler_noCmt import fetch_posts, fetch_content_only
import pandas as pd
import time
os.makedirs('./data', exist_ok=True)

BOARDS = {
    '마법사':    (2295, None),
    '궁수':      (2296, None),
    '도적':      (2297, None),
    '전사':      (2294, None),
    '해적':      (2298, None),
    '자유':      (5974, None),
    '자유_30추': (5974, 'chuchu'),
    '자유_10추': (5974, 'chu'),
}

POSTS_CSV = './data/posts.csv'

def load_collected_post_ids():
    if not os.path.exists(POSTS_CSV):
        return set()
    try:
        df = pd.read_csv(POSTS_CSV, dtype=str)
        return set(df['번호'].astype(str))
    except Exception:
        return set()

def crawl_all(max_pages_per_board=5):
    post_fields = ['게시판','번호','제목','링크','본문']
    collected_post_ids = load_collected_post_ids()
    post_file_exists = os.path.exists(POSTS_CSV)

    # 각 게시판별로 현재 순회중인 페이지번호 저장
    current_pages = {board: 51 for board in BOARDS.keys()}
    finished = set()

    with open(POSTS_CSV, 'a', newline='', encoding='utf-8-sig') as pf:
        pw = csv.DictWriter(pf, fieldnames=post_fields)
        if not post_file_exists:
            pw.writeheader()

        while len(finished) < len(BOARDS):
            for board_name, (bid, flt) in BOARDS.items():
                if board_name in finished:
                    continue
                page_start = current_pages[board_name]
                page_end = page_start + max_pages_per_board - 1
                empty_page_hit = False
                for page in range(page_start, page_end + 1):
                    posts = fetch_posts(bid, page, flt)
                    if not posts:
                        finished.add(board_name)
                        empty_page_hit = True
                        break
                    print(f'[{board_name}] 페이지 {page} → {len(posts)}건')
                    for post in posts:
                        if post['번호'] in collected_post_ids:
                            continue
                        try:
                            content = fetch_content_only(post['링크'])
                        except Exception as e:
                            print(f"[ERROR] fetch_content_only 실패: {post['링크']} → {e}")
                            content = ''
                        pw.writerow({'게시판': board_name, '번호': post['번호'], '제목': post['제목'], '링크': post['링크'], '본문': content})
                        collected_post_ids.add(post['번호'])
                        time.sleep(2)
                if not empty_page_hit:
                    current_pages[board_name] += max_pages_per_board

    print('크롤링 완료 → data/posts.csv')

if __name__ == '__main__':
    crawl_all(max_pages_per_board=5)


[마법사] 페이지 51 → 40건
[마법사] 페이지 52 → 40건
[마법사] 페이지 53 → 40건
[마법사] 페이지 54 → 40건
[마법사] 페이지 55 → 40건
[궁수] 페이지 51 → 40건
[궁수] 페이지 52 → 40건
[궁수] 페이지 53 → 40건
[궁수] 페이지 54 → 40건
[궁수] 페이지 55 → 40건
[도적] 페이지 51 → 40건
[도적] 페이지 52 → 40건
[도적] 페이지 53 → 40건
[도적] 페이지 54 → 40건
[도적] 페이지 55 → 40건
[전사] 페이지 51 → 40건
[전사] 페이지 52 → 40건
[전사] 페이지 53 → 40건
[전사] 페이지 54 → 40건
[전사] 페이지 55 → 40건
[해적] 페이지 51 → 40건
[해적] 페이지 52 → 40건
[해적] 페이지 53 → 40건
[해적] 페이지 54 → 40건
[해적] 페이지 55 → 40건
[자유] 페이지 51 → 70건
[자유] 페이지 52 → 70건
[자유] 페이지 53 → 70건
[자유] 페이지 54 → 70건
[자유] 페이지 55 → 70건
[자유_30추] 페이지 51 → 70건
[자유_30추] 페이지 52 → 70건
[자유_30추] 페이지 53 → 70건
[자유_30추] 페이지 54 → 70건
[자유_30추] 페이지 55 → 70건
[자유_10추] 페이지 51 → 70건
[자유_10추] 페이지 52 → 70건
[자유_10추] 페이지 53 → 70건
[자유_10추] 페이지 54 → 70건
[자유_10추] 페이지 55 → 70건
[마법사] 페이지 56 → 40건
[마법사] 페이지 57 → 40건
[마법사] 페이지 58 → 40건
[마법사] 페이지 59 → 40건
[마법사] 페이지 60 → 40건
[궁수] 페이지 56 → 40건
[궁수] 페이지 57 → 40건
[궁수] 페이지 58 → 40건
[궁수] 페이지 59 → 40건
[궁수] 페이지 60 → 40건
[도적] 페이지 56 → 40건
[도적] 페이지 57 → 40건
[도적] 페이지 58 → 

KeyboardInterrupt: 